In [ ]:
# from keras.models import load_model
from time import sleep
# from keras.preprocessing.image import img_to_array
# from keras.preprocessing import image
import keras
import cv2
import numpy as np
from keras import models
import tensorflow as tf

#첨부파일 중 xml파일인데 굳이 다운받을 필요 없음
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')
#첨부파일 중 modelh5
classifier =models.load_model("/Users/krx/Desktop/facial_expression/model_filter.h5") #개인 경로

emotion_labels = ['Angry','Disgust','Fear','Happy','Neutral', 'Sad', 'Surprise']

#노트북 웹캠을 카메라로 사용(colab에서는 실행 안됨)
cap = cv2.VideoCapture(0)

cap.set(3,640) # 너비
cap.set(4,480) # 높이

while cv2.waitKey(50) != ord('a'):#a를 누르면 멈춤
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)#좌우대칭
    labels = []
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray,1.05, 5)
    print(len(faces)) # 삭제해도 되는 코드 
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        roi_gray = gray[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)

        if np.sum([roi_gray])!=0:
            roi = roi_gray.astype('float')/255.0
            roi = tf.keras.preprocessing.image.img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)

            prediction = classifier.predict(roi)[0]
            label=emotion_labels[prediction.argmax()]
            label_position = (x,y)
            cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
        else:
            cv2.putText(frame,'No Faces',(30,80),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    cv2.imshow('Emotion Detector',frame)
    
#메모리 해제 method "cap.release()"를 사용해 카메라 장치에서 받아온 메모리를 해제
cap.release()
#모든 윈도우창 제거 함수
cv2.destroyAllWindows()